In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

## EDA & FE

타겟변수 확인

In [ ]:
target = train['count']

In [ ]:
from scipy import stats
from scipy.stats import norm

In [ ]:
sns.distplot(train['count'],fit=norm)

In [ ]:
stats.probplot(train['count'], plot = plt)

왼쪽으로 치우친 형태를 보이고 있으므로 멱변환이 필요할거 같다.
또한 PP플롯을 확인해 본 결과 정규성을 띄지 않으므로 박스칵스 변환을 한다.

In [ ]:
train["log_count"] = np.log(target+1)

In [ ]:
sns.distplot(train["log_count"], fit=norm)

In [ ]:
stats.probplot(train["log_count"], plot = plt)

박스칵스 변환 후에 왜도편중을 조금이나마 줄인거같다,.

In [ ]:
feature_names=list(test)
df_train=train[feature_names]
df=pd.concat((df_train, test))

In [ ]:
print(train.shape, test.shape, df.shape)

데이터 전처리를 편리하게 하기 위해서 train데이터와 test 데이터를 합쳤다.


In [ ]:
df.head()

In [ ]:
df.info()

datetime

In [ ]:
import datetime

In [ ]:
tmp = pd.to_datetime(train['datetime'])

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['weekend'] = (df['dayofweek'] ==5) | (df['dayofweek'] == 6)

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'])
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek
train['month'] = train['datetime'].dt.month
train['year'] = train['datetime'].dt.year
train['weekend'] = (train['dayofweek'] ==5) | (train['dayofweek'] == 6)

In [ ]:
df.drop(['datetime'], axis=1, inplace=True)

In [ ]:
figure, axs = plt.subplots(3,2, figsize = (15,10))

sns.barplot(data=train, x = "day", y = target, ax = axs[0][0])
sns.barplot(data=train, x = "hour", y = target, ax = axs[0][1])
sns.barplot(data=train, x = "dayofweek", y = target, ax = axs[1][0])
sns.barplot(data=train, x = "weekend", y = target, ax = axs[1][1])
sns.barplot(data=train, x = "month", y = target, ax = axs[2][0])
sns.barplot(data=train, x = "year", y = target, ax = axs[2][1])

2012년도에 대여량이 늘어난 것을 볼 수 있습니다.

월별 대여량을 보면 여름에 대여량이 높은 것으로 볼 수 있습니다.

시간대별로는 출퇴근 시간에 대여량이 많은 것으로 보입니다.

월별과 일별 대여량은 별다른 특징이 없으므로 드롭해줍니다.

In [ ]:
df=df.drop(columns=['month', 'day'])

In [ ]:
df

season

In [ ]:
sns.barplot(data=df[:len(train)], x='season', y=target)

In [ ]:
season_encoded = pd.get_dummies(df['season'],prefix= 'season')
df = pd.concat((df,season_encoded), axis=1)
df = df.drop(columns = 'season')

holiday

In [ ]:
sns.barplot(data=df[:len(train)], x='holiday', y=target)

holiday 피처의 데이터로 보아서 카테고리 데이터로 봐야 할거같다.

In [ ]:
df['holiday'] = df['holiday']

Workingday

In [ ]:
sns.barplot(data=df[:len(train)], x='workingday', y=target)

In [ ]:
df['workingday'] = df['workingday']

workingday 피처의 데이터로 보아서 카테코리 데이터로 봐야할거 같다

weather

In [ ]:
sns.barplot(data=df[:len(train)], x='weather', y=target)
df['weather'] = df['weather']

In [ ]:
weather_encoded = pd.get_dummies(df['weather'],prefix= 'weather')
df = pd.concat((df,weather_encoded), axis=1)
df = df.drop(columns = 'weather')

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows = 5)
fig.set_size_inches(20,30)

sns.pointplot(data = train, x = "hour", y = "count", ax = ax1)
sns.pointplot(data = train, x = "hour", y = "count", hue = "season", ax = ax2)
sns.pointplot(data = train, x = "hour", y = "count", hue = "holiday", ax = ax3)
sns.pointplot(data = train, x = "hour", y = "count", hue = "workingday", ax = ax4)
sns.pointplot(data = train, x = "hour", y = "count", hue = "weather",  ax = ax5)

변수들을 pointplot을 그려주었습니다.
첫번째 그래프는 출퇴근 시간에 대여량이 높은 것을 볼 수 있습니다.

두번째 그래프는 봄에 대여량이 낮을 것을 볼 수 있습니다.

세번째 그래프는 쉬는날에는 출퇴근 시간에도 대여량이 높지 않은 것을 볼 수 있습니다.

마지막 그래프는 맑은날 가장 높은 대여량을 보이며 날이 안좋을 때에는 대여량이 거의 없다고 봐도 될거같습니다.

왜도 확인

In [ ]:
from scipy.stats import skew
skew = df.apply(lambda x: skew(x))
skew.sort_values(ascending = False)

In [ ]:
skew = skew[abs(skew) > 0.5]
skew

상관분석

In [ ]:
cor = train.iloc[:,1:-1].corr()
cor.head()

In [ ]:
mask = np.array(cor)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(cor,mask= mask,square=True,annot=True)

temp와 humidity 피처 사이에 강한 상관관계를 가지고 있기 때문에 다중공선성을 확인해 봐야할거같다.
casual과 registered 피처는 누설변수 이기 때문에 고려하지 않는다.
타겟변수인 cout와 temp,humidity,windspeed를 고려해야 할거 같다.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif_data = df.iloc[:,:6]
vif_data.info()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = vif_data.columns
vif['vif'] = [variance_inflation_factor(
             vif_data.values, i) for i in range(vif_data.shape[1])]
vif.sort_values(by='vif',ascending=False)

vif가 10이상이면 다중공선성이 존재한다고 하는데 atemp와 temp에 다중공선성 문제가 보임
주성분 분석으로 차원을 줄여보기로 함

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=1)
pca.fit(df[['temp', 'atemp']])

In [ ]:
pca.explained_variance_ratio_

In [ ]:
df['pca']=pca.fit_transform(df[['temp','atemp']])

In [ ]:
sns.distplot(df['pca'], fit=norm)

distplot 결과 파생변수인 pca는 정규분포를 따르고 있다고 본다.

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(1,3)
fig.set_size_inches(12,5)
sns.regplot(train['temp'], 'count', data = train, ax=ax1)
sns.regplot(train['humidity'], 'count', data = train, ax=ax2)
sns.regplot(train['windspeed'], 'count', data = train, ax=ax3)

In [ ]:
stats.pearsonr(train['temp'],target)

세 변수 regplot을 확인해본 결과 windspeed의 경우 0쪽에 숫자가 몰려있는거 같다.

In [ ]:
sns.countplot(data = df, x = "windspeed")

풍속의 경우 0에 데이터가 많이 몰려있는 것으로 보이므로 피처를 조정해 줘야 할 거 같다.

windspeed 데이터 예측

In [ ]:
df.loc[df['windspeed']==0, 'windspeed']=df['windspeed'].mean()

RF를 통해서 풍속이 0인 구간을 예측하였으므로 앞에서 PCA를 했기 때문에 temp와 atemp 피처는 드롭하였다.

In [ ]:
df = df.drop(columns=['temp','atemp'])

범주형 변수

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize = (15,20))
sns.boxplot(data = train, y="count", x = "holiday", orient = "v", ax = axes[0][0])
sns.boxplot(data = train, y="count", x = "workingday", orient = "v", ax = axes[0][1])
sns.boxplot(data = train, y="count", x = "hour", orient = "v", ax = axes[1][0])
sns.boxplot(data = train, y="count", x = "dayofweek", orient = "v", ax = axes[1][1])
sns.boxplot(data = train, y="count", x = "year", orient = "v", ax = axes[2][0])

앞에서 보았던 특징들과 대부분 일치하는 특징을 보였다.

## 모델링

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression

train_test_split

In [ ]:
new_train = df[:train.shape[0]]
new_test = df[train.shape[0]:]

In [ ]:
target = train['log_count']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(new_train, target, test_size=0.2, shuffle=True)

검증을 하기 위해서 data 데이터를 train_test_split을 이용해서 trian과 validation으로 나누었다

RMSLE Score

In [ ]:
def rmsle_score(preds, true):
    rmsle_score = (np.sum((np.log1p(preds)-np.log1p(true))**2)/len(true))**0.5
    return rmsle_score

In [ ]:
from sklearn.metrics.scorer import make_scorer

RMSLE = make_scorer(rmsle_score)

단순선형회귀

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.OLS(target.values, new_train.astype(float))

In [ ]:
re = model.fit()

In [ ]:
re.summary()

모델의 설명력이 낮으며 p-value가 높은 값들이 존재하며 다중공선성의 문제가 있을 수 도 있다.

Lasso

In [ ]:
param = {'alpha':[1e-03,0.01,0.1,0.5,0.8,1], 'normalize':[True,False], 'tol':[1e-06,1e-05,5e-05,1e-04,5e-04,1e-03]}

In [ ]:
lasso = make_pipeline(GridSearchCV(Lasso(random_state=1),param,
                                  cv=10, scoring = RMSLE))

In [ ]:
lasso.fit(X_train,y_train)

In [ ]:
la_yhat = lasso.predict(X_val)

In [ ]:
s_lasso = rmsle_score(la_yhat,y_val)
s_lasso

In [ ]:
pred_la = lasso.predict(new_test)

ElasticNet

In [ ]:
param_e = {'alpha' :[0.1,1.0,10], 'max_iter' :[1000000], 'l1_ratio':[0.04,0.05], 
           'normalize':[True,False]}

In [ ]:
Enet = make_pipeline(GridSearchCV(ElasticNet(random_state=1),param_e,
                     cv=10, scoring = RMSLE))

In [ ]:
Enet.fit(X_train,y_train)

In [ ]:
Enet_yhat = Enet.predict(X_val)

In [ ]:
s_Enet = rmsle_score(Enet_yhat,y_val)
s_Enet

In [ ]:
pred_Enet = Enet.predict(new_test)

RandomForestRegressor

In [ ]:
param_Rf =  {'min_samples_split' : [3,4,6,10], 'n_estimators' : [70,100], 'random_state': [5] }

In [ ]:
RF = make_pipeline(GridSearchCV(RandomForestRegressor(random_state=1),param_Rf,
                   cv=10, scoring = RMSLE))

In [ ]:
RF.fit(X_train,y_train)

In [ ]:
RF_yhat = RF.predict(X_val)
s_RF = rmsle_score(RF_yhat,y_val)
s_RF

In [ ]:
pred_RF = RF.predict(new_test)

GradientBoostingRegressor

In [ ]:
param_GB = [{'learning_rate': [1,0.1,0.01,0.001],
              'n_estimators': [50, 100, 200, 500, 1000]}]

In [ ]:
GB = make_pipeline(GridSearchCV(GradientBoostingRegressor(random_state=1),param_GB,
                   cv=10, scoring = RMSLE))

In [ ]:
GB.fit(X_train,y_train)

In [ ]:
GB_yhat = GB.predict(X_val)
s_GB = rmsle_score(GB_yhat,y_val)
s_GB

In [ ]:
pred_GB = GB.predict(new_test)

XGBRegressor

xgb의 경우 라벨인코더로 예측할수 있지만 성능이 떨어지므로 모델링을 하지 않기로 했다.

param_xgb = [{'learning_rate': [1,0.1,0.01,0.001],
              'n_estimators': [50, 100, 200, 500, 1000],
              'max_depth' : [1,3,5,10,50]}]

xgb = make_pipeline(GridSearchCV(XGBRegressor(silent=True,random_state=1),param_xgb,
                    cv=10, scoring = RMSLE))

xgb.fit(X_train,y_train)

xgb_yhat = xgb.predict(X_val)
s_xgb = rmsle_score(xgb_yhat,y_val)
s_xgb

pred_xgb = xgb.predict(new_test)

LGBMRegressor

In [ ]:
param_lgb = param_grid = [{
    'n_estimators': [400, 700, 1000], 
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
    'min_split_gain': [0.3, 0.4],
}]

In [ ]:
lgb = make_pipeline(GridSearchCV(LGBMRegressor(verbose_eval=False,random_state=1),param_lgb,
                    cv=10, scoring = RMSLE))

In [ ]:
lgb.fit(X_train,y_train)

In [ ]:
lgb_yhat = lgb.predict(X_val)
s_lgb = rmsle_score(lgb_yhat,y_val)
s_lgb

In [ ]:
pred_lgb = lgb.predict(new_test)

In [ ]:
list_scores = [s_lasso, s_Enet, s_RF,s_GB,s_lgb]
list_regressors = ['Lasso','Enet','RF','GB','lgb']

In [ ]:
sns.barplot(x=list_regressors, y=list_scores)
plt.ylabel('RMSE')

앙상블

In [ ]:
predictions = {'Lasso': pred_la,
               'ElaNet': pred_Enet, 
               'RF': pred_RF,
               'GB': pred_GB,
               #'XGB' : pred_xgb,
               'lgb' : pred_lgb
              }

In [ ]:
df_predictions = pd.DataFrame(data=predictions) 
df_predictions.corr()

In [ ]:
plt.figure(figsize=(7, 7))
sns.heatmap(df_predictions.corr(),linewidths=1.5,
            annot=True, 
            square=True,          
            yticklabels=df_predictions.columns , 
            xticklabels=df_predictions.columns)


lgb모델과 RF모델이 상관관계가 높게 나오므로 앙상블을 통해 예측을 하기로 한다.

## Pred

RF 모델 예측

In [ ]:
RF.fit(new_train,target)

In [ ]:
log_pred=RF.predict(new_test)
predictions=np.exp(log_pred)-1

In [ ]:
sub = pd.DataFrame()
sub['datetime'] = test['datetime']
sub['count'] = predictions
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)

이 경우 RMSE score가 0.39182가 나왔습니다.

lgb 모델 예측

In [ ]:
lgb.fit(new_train,target)

In [ ]:
log_pred_lgb=lgb.predict(new_test)
predictions_lgb=np.exp(log_pred_lgb)-1

In [ ]:
sub = pd.DataFrame()
sub['datetime'] = test['datetime']
sub['count'] = predictions_lgb
sub.head()

이 경우 RMSE score가 0.39461가 나왔습니다.

In [ ]:
ensemble = (0.6*predictions + 0.4*predictions_lgb) 

In [ ]:
sub = pd.DataFrame()
sub['datetime'] = test['datetime']
sub['count'] = ensemble
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)

앙상블로 모델을 만들어 예측 결과 RMSE score가 0.38762 나왔습니다.